In [24]:
import datetime

filename = 'moonphasedf.csv'

def build_phases(filename):
    phases = {}
    f = open(filename)
    lines = f.readlines()
    f.close() 
    
    for i in range(1,len(lines)):
        try:
            date = datetime.datetime.strptime(lines[i].split(',')[1], '%Y %b %d')
            value = lines[i].split(',')[2]
        except Exception as e:
            print(e)
            continue

        if value not in phases:
            phases[value] = []
        phases[value].append(date)
        
    f.close()  
    
    return phases

def build_phases_from_string(filename):
    phases = {}
    f = open(filename)
    lines = f.readlines()
    f.close() 
    
    one_day = datetime.timedelta(days=1)
    six_days = datetime.timedelta(days=6)
    
    for i in range(1,len(lines)):
        try:
            date, value, time = lines[i].split(',')[1], lines[i].split(',')[2], lines[i].split(',')[3]
            yyyy, mmm, dd = date.split(' ')[0], date.split(' ')[1], date.split(' ')[2]
            hh, mm = time.split(':')[0], time.split(':')[1]
            mm = mm.strip()
            timestamp = datetime.datetime.strptime(date + ' ' + time, '%Y %b %d %H:%M\n')

        except Exception as e:
            print(e)
            continue
        
        adjusted_time = timestamp - one_day if ((value == 'Full Moon') | (value == 'New Moon')) else timestamp - six_days
        s = adjusted_time.strftime('%Y %b %d %H:%M\n')
#         s = timestamp.strftime('%Y %b %d %H:%M\n')
        yyyy, mmm, dd, time = s.split(' ')[0], s.split(' ')[1], s.split(' ')[2], s.split(' ')[3]
        hh, mm = time.split(':')[0], time.split(':')[1]
        mm = mm.strip()
        
        if yyyy not in phases:
            phases[yyyy] = {}
        if mmm not in phases[yyyy]:
            phases[yyyy][mmm] = {}
        if dd not in phases[yyyy][mmm]:
            phases[yyyy][mmm][dd] = {}
        if hh not in phases[yyyy][mmm][dd]:
            phases[yyyy][mmm][dd][hh] = {}
        phases[yyyy][mmm][dd][hh][mm] = value
        

    f.close()  
    
    return phases

def get_previous_phase(phase, direction):
    phase_strings = ['New Moon', 'First Quarter', 'Full Moon', 'Last Quarter']
    i = phase_strings.index(phase)
    return phase_strings[(i+direction)%4]

def get_phase_by_hour(dict_):
    minutes = list(dict_.keys())[0]
    return dict_[minutes]

def get_phase_by_day(dict_):
    hour = list(dict_.keys())[0]
    return get_phase_by_hour(dict_[hour])
    

def get_phase_from_day(day_string):
    try:
        elements = day_string.split(' ')
        yyyy = elements[0]
        mmm = elements[1]
        dd = elements[2]
        hh,mm = elements[3].split(':')
    except Exception as e:
        print(e)
        
    phase = 'No Phase'
    if yyyy in phases:
        if mmm in phases[yyyy]:
            if dd in phases[yyyy][mmm]:
                if hh in phases[yyyy][mmm][dd]:
                    if mm in phases[yyyy][mmm][dd][hh]:
                        phase = phases[yyyy][mmm][dd][hh][mm]
                    else:
                        minutes = list(phases[yyyy][mmm][dd][hh].keys())[0]
                        if int(mm) > int(minutes):
                            phase = phases[yyyy][mmm][dd][hh][minutes]
                        else:
                            phase = get_previous_phase(phases[yyyy][mmm][dd][hh][minutes], -1)
                else:
                    hours = list(phases[yyyy][mmm][dd].keys())[0]
                    phase = get_phase_by_hour(phases[yyyy][mmm][dd][hours])
                    if int(hh) < int(hours):
                        phase = get_previous_phase(phase, -1)
                    else:
                        print('hours', hh)
            else:
                days = list(phases[yyyy][mmm].keys())
                for day in days:
                    if int(day) > int(dd):
                        phase = get_phase_by_day(phases[yyyy][mmm][day])
                        phase = get_previous_phase(phase, -1)
                        break
                    else:
                        max_day = day
                if int(dd) > int(max_day):
                    phase = get_phase_by_day(phases[yyyy][mmm][max_day])
        else:
            print('Month', mmm)
    else:
        print('year', yyyy)
                                    
    return phase
                        
                    
    
someday = '2017 Sep 19 05:29'
phases = build_phases_from_string(filename)    
get_phase_from_day(someday)

'Last Quarter'